In [4]:
#!/usr/bin/env python3
"""
fetch_qqq_contracts_chain.py

Fetch the full QQQ options chain for CALLs on a given snapshot date
using the `/options/contracts` endpoint, compute true DTE from `exp_date`,
sort by ascending DTE, and write to CSV. Handles 422/404 as end-of-data.
"""

import requests
import pandas as pd
from datetime import datetime
import sys
import time

# ─── CONFIGURATION ─────────────────────────────────────────────────────────────
API_TOKEN = "67fb3d6f50c489.92544905"       # Your EODHD API token
SYMBOL    = "QQQ"
SNAPSHOT  = "2025-04-16"                    # Snapshot date for chain
BASE_URL  = "https://eodhd.com/api/mp/unicornbay/options/contracts"
PAGE_LIMIT= 1000                            # Max per docs
PAUSE_SEC = 0.2                             # To respect rate limits (1000 req/min)

# ─── FETCH FULL CHAIN VIA /options/contracts ──────────────────────────────────
def fetch_contracts_for_date(trade_date):
    all_records = []
    offset = 0

    while True:
        params = {
            "api_token":                   API_TOKEN,
            "filter[underlying_symbol]":   SYMBOL,
            "filter[date_eq]":             trade_date,
            "filter[type]":                "call",
            "page[limit]":                 PAGE_LIMIT,
            "page[offset]":                offset,
            "sort":                        "exp_date",  # nearest expirations first
            "compact":                     "0",
        }
        resp = requests.get(BASE_URL, params=params)
        if resp.status_code in (422, 404):
            print(f"Reached end of data at offset {offset} (status {resp.status_code}).")
            break
        try:
            resp.raise_for_status()
        except requests.HTTPError:
            print(f"HTTP error @ offset {offset}: {resp.status_code}", file=sys.stderr)
            sys.exit(1)

        data = resp.json().get("data", [])
        if not data:
            break

        all_records.extend(data)
        print(f"Fetched {len(data)} rows @ offset {offset} (total={len(all_records)})")
        if len(data) < PAGE_LIMIT:
            break

        offset += PAGE_LIMIT
        time.sleep(PAUSE_SEC)

    return all_records

# ─── MAIN ──────────────────────────────────────────────────────────────────────
def main():
    print(f"📡 Fetching full CALL chain for {SYMBOL} on {SNAPSHOT}…")
    records = fetch_contracts_for_date(SNAPSHOT)
    if not records:
        print("❌ No chain data returned—check API key/date.", file=sys.stderr)
        sys.exit(1)

    df = pd.json_normalize(records)
    df["exp_date"] = pd.to_datetime(df["attributes.exp_date"], errors="coerce")
    df["trade_dt"] = pd.to_datetime(SNAPSHOT)
    df["dte"]      = (df["exp_date"] - df["trade_dt"]).dt.days
    df["dte"]      = pd.to_numeric(df["dte"], errors="coerce")

    df_sorted = df.sort_values("dte")
    output = f"{SYMBOL}_chain_{SNAPSHOT}.csv"
    df_sorted.to_csv(output, index=False)
    print(f"✅ Wrote {len(df_sorted)} contracts to {output}")

if __name__ == "__main__":
    main()


📡 Fetching full CALL chain for QQQ on 2025-04-16…
Fetched 1000 rows @ offset 0 (total=1000)
Fetched 1000 rows @ offset 1000 (total=2000)
Fetched 1000 rows @ offset 2000 (total=3000)
Fetched 1000 rows @ offset 3000 (total=4000)
Fetched 1000 rows @ offset 4000 (total=5000)
Fetched 1000 rows @ offset 5000 (total=6000)
Fetched 1000 rows @ offset 6000 (total=7000)
Fetched 1000 rows @ offset 7000 (total=8000)
Fetched 1000 rows @ offset 8000 (total=9000)
Fetched 1000 rows @ offset 9000 (total=10000)
Fetched 1000 rows @ offset 10000 (total=11000)
Reached end of data at offset 11000 (status 422).
✅ Wrote 11000 contracts to QQQ_chain_2025-04-16.csv
